In [6]:
from binance.spot import Spot as sp
import inspect as i
import json
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import pytz
from plotly.subplots import make_subplots
from finta import TA as ta
import os

tz_local = pytz.timezone("Europe/Zurich")

apiKey = "OY6oBQD3qm6VGzokI29UaCm8hrnpCrjv4KEpCY45UasLAj7D3bbxkLPGwFjNNz1neL"
apiKey_secret = "D82a2gKS5lGbmFRvkPP9PEQTwnfXKfxFBNddOVrjixQ8MJkSirLxuRyNoenO3JVj"

baseURL = "https://testnet.binance.vision"

pd.options.display.float_format = "{:,.3f}".format

In [7]:
df_csv = pd.read_csv("./app/binance_ws_comp2.csv")
df_csv["timestamp"] = pd.to_datetime(df_csv["timestamp"])
df_csv.reset_index(drop=True, inplace=True)
df_csv.set_index("timestamp", inplace=True)

df_csv.index = df_csv.index.tz_localize("UTC").tz_convert("CET")
df_csv.index.max()

Timestamp('2021-08-22 15:20:01+0200', tz='CET')

In [8]:
start_time = df_csv.index.min().floor("min")
end_time = df_csv.index.max().ceil("min")



start_ms = int(round(start_time.timestamp()*1000,0))
end_ms = int(round(end_time.timestamp()*1000,0))

end_time

Timestamp('2021-08-22 15:21:00+0200', tz='CET')

In [9]:
#API SETUP
sp_client = sp(key=apiKey, secret=apiKey_secret)

#get Different cryptos
dict_symbols = sp_client.exchange_info()

#API CALL
api_interval = "1m"
pd_interval = "T"
api_limit = 1000

dict_data = sp_client.klines(startTime=start_ms , endTime=end_ms, symbol="BTCBUSD", interval="1m")
df_symbols = pd.json_normalize(dict_symbols["symbols"])
headers_df = ["openTime","open", "high","low","close","vol","closeTime","quiteAssetVolume","trades_qty","baseAssetVol_taker","baseQuoteVol_taker","ignore"]


In [10]:
headers_df = ["openTime","open", "high","low","close","vol","closeTime","quiteAssetVolume","trades_qty","baseAssetVol_taker","baseQuoteVol_taker","ignore"]

df_data_raw = pd.DataFrame(dict_data, columns=headers_df)
#convert to datetimes
df_data_raw["openTime"] = pd.to_datetime(df_data_raw["openTime"]/1000, unit="s")
df_data_raw["closeTime"] = pd.to_datetime(df_data_raw["closeTime"]/1000, unit="s")

#round to nearest second
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.floor("S")

#change timezone

#reset index to opentime
df_data_raw.reset_index(inplace=True,drop=True)
df_data_raw.set_index("openTime", inplace=True)

# convert all none-date columns to float
cols = df_data_raw.select_dtypes(exclude=["datetime64"]).columns
df_data_raw[cols] = df_data_raw[cols].apply(pd.to_numeric, downcast="float",errors="coerce")

# mark TZ as UTC then change to CET
df_data_raw.index = df_data_raw.index.tz_localize("UTC").tz_convert("CET")
df_data_raw["closeTime"] = df_data_raw["closeTime"].dt.tz_localize("UTC").dt.tz_convert("CET")


In [11]:
df_d = df_data_raw[["open","high","low","close"]].copy()
df_d.index.max()

Timestamp('2021-08-22 15:21:00+0200', tz='CET')

In [12]:
r_int = 5

df_csv_r = df_csv.resample("{}S".format(r_int), closed="left").mean()
df_csv_r.head()

,open,high,low,close,bid,ask
timestamp,,,,,,
2021-08-22 14:20:00+02:00,"48,660.010","49,757.040","48,562.800","49,021.303","49,021.300","49,021.310"
2021-08-22 14:20:05+02:00,"48,652.864","49,757.040","48,562.800","49,018.450","49,018.442","49,018.452"
2021-08-22 14:20:10+02:00,"48,650.008","49,757.040","48,562.800","49,014.568","49,014.562","49,014.572"
2021-08-22 14:20:15+02:00,"48,650.010","49,757.040","48,562.800","49,014.550","49,014.550","49,014.560"
2021-08-22 14:20:20+02:00,"48,651.446","49,757.040","48,562.800","49,012.698","49,012.696","49,012.706"


In [13]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        mode ="lines"
        , x=df_csv.index
        , y=df_csv["close"]
        , line = dict(color="blue", width=1)
        , name="WS_data"
    )
)

fig.add_trace(
    go.Scatter(
        mode ="lines"
        , x=df_csv_r.index + pd.Timedelta(seconds=r_int)
        , y=df_csv_r["close"]
        , line = dict(color="green")
        , name="WS_data"
    )
)


fig.add_trace(
    go.Scatter(
        mode="lines+markers"
        , x=df_d.index + pd.Timedelta(minutes=1)
        , y=df_d["close"]
        , line = dict(color="red")
        , name="WS_data"
    )
)

fig.update_layout(
    width=1200
    , height=500
)

fig.show()